# Data Preprocessing

In [1]:
import re
from pathlib import Path
import pandas as pd 
import numpy as np
from ecbutil import *

In [2]:
dir_data = Path("../data")
in_path = dir_data / "ECBDataset_270519b.csv"
out_path = dir_data / "ECBDataset_150819.csv"
# fields to add from metadata for speakers
speaker_fields1 = ["sp_nationality", "sp_fieldofstudy", "sp_phd", "sp_academia" ]
speaker_fields2 = ["sp_banking_finance", "sp_centralbank", "sp_govtmember_staterep", 
                   "sp_imf", "sp_wb", "sp_yearsacademia", "sp_yearsbanking_finance", "sp_yearscentralbank", "sp_yearsgovtmember_staterep", "sp_yearsimf", "sp_yearswb"]
extra_speaker_fields = speaker_fields1 + speaker_fields2

In [3]:
print("Reading %s" % in_path)
if in_path.suffix == ".csv":
    df = pd.read_csv( in_path, sep=",", low_memory=False )
elif in_path.suffix == ".txt":
    df = pd.read_csv( in_path, sep="\t", low_memory=False)
elif in_path.suffix == ".xlsx":
    df = pd.read_excel( in_path, sheet_name='Sheet1', low_memory=False )
print("Read %d rows" % len(df) )

Reading ../data/ECBDataset_270519b.csv
Read 101144 rows


Make sure we have all of the required metadata fields:

In [4]:
for field in extra_speaker_fields:
    if not field in df.columns:
        print("Missing field", field)

Check for missing values:

In [5]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    col_dtypes = df.dtypes
    mis_val_table = pd.concat([mis_val, mis_val_percent,col_dtypes], axis=1)
    df_miss = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values', 2 : "kind"})
    df_miss = df_miss[
    df_miss.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    return df_miss

In [6]:
df_miss = missing_values_table(df)
df_miss

,Missing Values,% of Total Values,kind
SecuritiesSupervisor,100807,99.7,object
SecuritiesSupervisor2,98479,97.4,object
InsuranceSupervisor2,98321,97.2,object
BankingSupervisor2,94494,93.4,object
CBSecuritiesSupervisor,69639,68.9,float64
Country,69639,68.9,object
CBInsuranceSupervisor,69639,68.9,float64
CBBankSupervisor,69639,68.9,float64
DualSecuritiesSupervisor,69639,68.9,float64
DualInsuranceSupervisor,69639,68.9,float64


Fill in the missing values with an appropriate type of value:

In [7]:
for col, row in df_miss.iterrows():
    if row["kind"] == object:
        print("Column %s: Filling with strings" % col)
        df[[col]] = df[[col]].fillna("Unknown")
    elif row["kind"] == np.float64:
        print("Column %s: Filling with numbers" % col)
        df[[col]] = df[[col]].fillna(0)        

Column SecuritiesSupervisor: Filling with strings
Column SecuritiesSupervisor2: Filling with strings
Column InsuranceSupervisor2: Filling with strings
Column BankingSupervisor2: Filling with strings
Column CBSecuritiesSupervisor: Filling with numbers
Column Country: Filling with strings
Column CBInsuranceSupervisor: Filling with numbers
Column CBBankSupervisor: Filling with numbers
Column DualSecuritiesSupervisor: Filling with numbers
Column DualInsuranceSupervisor: Filling with numbers
Column DualBankSupervisor: Filling with numbers
Column BankingSupervisor1: Filling with strings
Column CentralBankName: Filling with strings
Column InsuranceSupervisor1: Filling with strings
Column SecuritiesSupervisor1: Filling with strings
Column ECBsubtitle: Filling with strings
Column ECBtitle: Filling with strings
Column sp_yearsacademia: Filling with numbers
Column sp_yearsgovtmember_staterep: Filling with numbers
Column sp_yearscentralbank: Filling with numbers
Column sp_yearsimf: Filling with nu

Store the pre-processed data file:

In [8]:
print("Writing %s" % out_path)
df.to_csv( out_path, sep="," )

Writing ../data/ECBDataset_150819.csv
